In [51]:
%load_ext autoreload
%autoreload 2

In [52]:
from dask.distributed import Client, LocalCluster

client = Client() # Note that `memory_limit` is the limit **per worker**.
# n_workers=4,
#                 threads_per_worker=1,
#                 memory_limit='3GB'
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [53]:
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import scipy.constants
import scipy.signal as signal
import pickle
import holoviews as hv

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

### Open and resave file

In [ ]:
# file path to data and configs

prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/drone/radar_data/20230723-summit-day3-2start/20230723_104248"

# resave data as zarr for dask processing
zarr_base_location="/home/thomas/Documents/StanfordGrad/RadioGlaciology/test_tmp_zarr_cache/"
zarr_path = pr.save_radar_data_to_zarr(prefix, zarr_base_location=zarr_base_location)

# open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1000})
#raw = pr.invert_phase_dithering(raw, phase_codes_filename='/Users/abroome/Documents/SDR/uhd_radar/phases_1M.bin')


### Enter processing parameters

In [ ]:
#zero_sample_idx = 36 # X310, fs = 20 MHz
zero_sample_idx = 63 # X310, fs = 50 MHz
#zero_sample_idx = 81 # X310, fs = 100 MHz ????????
#zero_sample_idx = 159 # B205mini, fs = 56 MHz
#zero_sample_idx = 166 # B205mini, fs = 20 MHz

nstack = 1 #len(raw.pulse_idx) # number of pulses to stack

modify_rx_window = False # set to true if you want to window the reference chirp only on receive, false uses ref chirp as transmitted in config file
rx_window = "kaiser14" # what you want to change the rx window to if modify_rx_window is true

dielectric_constant = 3.17 # ice (air = 1, 66% velocity coax = 2.2957)
#dielectric_constant = 2.2957 # COAX (air = 1, 66% velocity coax = 2.2957)
sig_speed = scipy.constants.c / np.sqrt(dielectric_constant)

### Generate reference chirp

In [ ]:
if modify_rx_window:
    config = copy.deepcopy(raw.config)
    config['GENERATE']['window'] = rx_window
else:
    config = raw.config
    
# config = copy.deepcopy(raw.config)
# config['GENERATE']['lo_offset_sw'] = 12.5e6

chirp_ts, ref_chirp = generate_chirp(config)

### Filter reference chirp (optional)

In [ ]:
filter = signal.butter(1, 0.5e6, btype='highpass', output='sos', fs=config['GENERATE']['sample_rate'])

ref_chirp = signal.sosfilt(filter, ref_chirp)

### View raw pulse in time domain to check for clipping

In [ ]:
plot1 = np.real(raw.radar_data[:,0]).hvplot.line(x='fast_time', label='Real') * np.imag(raw.radar_data[:,0]).hvplot.line(x='fast_time', label='Imaginary')

plot1 = plot1.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
plot1

### Clean and stack data

In [ ]:
stacked = pr.fill_errors(raw, error_fill_value=0.0) # fill receiver errors with 0s

stacked = pr.stack(raw, nstack) # stack 

### Filter LO out of received data (optional)

In [ ]:
filt_hp = signal.butter(1, 0.5e6, btype='highpass', output='sos', fs=config['GENERATE']['sample_rate'])

filtered = xr.apply_ufunc(
    lambda x: signal.sosfilt(filt_hp, x),
    stacked,
    dask="parallelized"
)

In [ ]:
freqs_filt, h = signal.sosfreqz(filt_hp, fs=config['GENERATE']['sample_rate'])

# Plot magnitude response of the filter
fig = plt.figure()
plt.plot(freqs_filt/(1e6), 20 * np.log10(abs(h)),
         'r', label='Bandpass filter', linewidth='2')
plt.xlabel('Frequency [MHz]', fontsize=20)
plt.ylabel('Magnitude [dB]', fontsize=20)
plt.title('Filter', fontsize=20)
plt.grid()

### Pulse compress data

In [ ]:
compressed = pr.pulse_compress(stacked, ref_chirp,
                               fs=stacked.config['GENERATE']['sample_rate'],
                               zero_sample_idx=zero_sample_idx,
                               signal_speed=sig_speed)

compressed_power = xr.apply_ufunc(
    lambda x: 20*np.log10(np.abs(x)),
    compressed,
    dask="parallelized"
)

### View 1D pulse compressed data

In [ ]:
plot1D = compressed_power.radar_data[0,:].compute().hvplot.line()
#plot1D = plot1D * compressed_power.radar_data[0,:].hvplot.line()
# relevant options: xlim(-80,1000)

plot1D = plot1D.opts(xlabel='Reflection Distance (m)', ylabel='Return Power (dB)', title=raw.basename)
#plot1D.opts(xlim=(-50,200), ylim=(-120, -40), show_grid=True)
plot1D

In [ ]:
# USING MATPLOTLIB
fig, ax = plt.subplots(1,1, figsize=(10,6), facecolor='white')
ax.plot(compressed_power.reflection_distance, compressed_power.radar_data[0,:])
ax.set_xlabel('Depth (m)')
ax.set_ylabel('Return Power (dB)')
ax.set_title(raw.basename)

fig.savefig(f'/Users/abroome/Desktop/Summit2023/figs/{raw.basename}_pulse_compressed_n{nstack}.png', dpi=300)

### View 2D pulse compressed data (radargram)

In [ ]:
# USING HOLOVIEWS (sometimes breaks)
plot2D = compressed_power.swap_dims({'pulse_idx': 'slow_time', 'travel_time': 'reflection_distance'}).hvplot.quadmesh(x='slow_time', cmap='inferno', flip_yaxis=True)
# relevant options: ylim=(100,-50), clim=(-90,-40)

plot2D.opts(xlabel='Slow Time (s)', ylabel='Depth (m)', clabel='Return Power (dB)')
#plot2D.opts(ylim=(-10, 70), clim=(-120, -40))
plot2D

In [ ]:
# USING MATPLOTLIB (sometimes takes a while)
fig, ax = plt.subplots(1,1, figsize=(10,6), facecolor='white')

p = ax.pcolormesh(compressed_power.slow_time, compressed_power.reflection_distance, compressed_power.radar_data.transpose(), shading='auto', cmap='inferno')
ax.invert_yaxis()
clb = fig.colorbar(p, ax=ax)
clb.set_label('Return Power (dB)')
ax.set_xlabel('Slow Time (s)')
ax.set_ylabel('Distance to Reflector (m)')
ax.set_title(f'{raw.basename}: nstack={nstack}')
# relevant options: ax.set_ylim(100,-50), ax.set_xlim(0, 1), vmin=-90, vmax=40
ax.set_ylim(1500,-50)